# Parameters

They are the same from **GWP1** as requested:

$S_0 = 100$

$K = 100$ (because the question says *ATM*)

$r = 0.05$

$\sigma = 0.2$

$T = 0.25$ *years* (3 months)

# Step 1

## Team Member A (Atakan Devrent)

## 1)

In [54]:
import numpy as np
import scipy.stats as ss

# Black-Scholes closed-form solution for European option pricing
def BS_price(S0, K, r, sigma, T, t, call_or_put="C"):
    T = T-t
    d1 = (np.log(S0/K) + (r + sigma**2 / 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    if call_or_put == "C":
        price = S0 * ss.norm.cdf(d1) - np.exp(-r*T) * K * ss.norm.cdf(d2)
    else:
        price = np.exp(-r*T) * K * ss.norm.cdf(-d2) - S0 * ss.norm.cdf(-d1)
        
    return(np.round(price, 2))

In [55]:
S0 = 100
K = 100
r = 0.05
sigma = 0.2
T = 0.25

eu_call_price = BS_price(S0, K, r, sigma, T, 0, "C")
eu_put_price = BS_price(S0, K, r, sigma, T, 0, "P")
print("European ATM call price =", eu_call_price)
print("European ATM put price =", eu_put_price)

European ATM call price = 4.61
European ATM put price = 3.37


$\Delta_{call} = \Phi(d_1)$

$\Delta_{put} = \Phi(d_1) - 1$

In [56]:
d1 = (np.log(S0/K) + (r + sigma**2 / 2) * T) / (sigma * np.sqrt(T))

delta_call = np.round(ss.norm.cdf(d1), 2)
delta_put = np.round(ss.norm.cdf(d1) - 1, 2)

print("European ATM call delta =", delta_call)
print("European ATM put delta =", delta_put)

European ATM call delta = 0.57
European ATM put delta = -0.43


$\nu = S \times \Phi(d1) \times \sqrt{T - t} $

In [57]:
vega_call = np.round(S0 * ss.norm.cdf(d1) * np.sqrt(T - 0), 2)
vega_put = np.round(S0 * ss.norm.cdf(d1) * np.sqrt(T - 0), 2)

print("European ATM call vega =", vega_call)
print("European ATM put vega =", vega_put)

European ATM call vega = 28.47
European ATM put vega = 28.47


## Team Member B

# 2)  # TODO

## Team Member C

# 3)  # TODO

_________________________

# Step 2

## Team Member A (Atakan Devrent)

## 4)

In [58]:
import numpy as np

def MC_American_price(S0, K, r, sigma, T, N, M, t, call_or_put="C"):
    """Calculates the price of an American option by
    simulating price paths with geometric Brownian motion.

    Args:
        S0 (float): Initial stock price
        K (float): Strike price of the option
        r (float): Risk-free interest rate
        sigma (float): Volatility of the underlying stock
        T (float): Expiry time of the option
        N (int): Number of steps
        M (int): Number of different paths
        t (float): Current time
        call_or_put (str, optional): Type of the option. Defaults to "C".

    Returns:
        opt_price: Price of the American option at time t.
    """
    T = T - t
    dt = T / N
    S = np.zeros((N + 1, M))
    S[0] = S0  # at t=0, all prices are S0. (first row is S0 repeated.)
    rn = np.random.standard_normal((N, M))
    multipliers = np.exp((r - sigma**2 / 2) * dt + sigma * np.sqrt(dt) * rn)
    S[1:, :] = S0 * multipliers.cumprod(axis=0)  # price paths are generated

    # Initialize payoff array
    payoff = np.zeros((N + 1, M))
    
    # Final payoff at maturity
    if call_or_put == "C":
        payoff[-1] = np.maximum(S[-1] - K, 0)
    else:
        payoff[-1] = np.maximum(K - S[-1], 0)
    
    # Backward induction for American option
    for i in range(N - 1, 0, -1):
        # Discounted expected continuation value
        continuation_value = np.exp(-r * i * dt) * payoff[i + 1]
        
        # Immediate exercise value
        if call_or_put == "C":
            exercise_value = np.maximum(S[i] - K, 0)
        else:
            exercise_value = np.maximum(K - S[i], 0)
        
        # The holder will choose the maximum of continuing or exercising
        payoff[i] = np.maximum(exercise_value, continuation_value)
    
    # Discount the payoff from the first time step to the present
    opt_price = np.exp(-r * T) * payoff[1].mean()
    
    return np.round(opt_price, 2)

In [59]:
S0 = 100
K = 100
r = 0.05
sigma = 0.2
T = 0.25

N = 365  # number of steps are the total days in a year.
M = 10000  # generate as many price paths as possible to converge to the analytical solution

american_call_price = MC_American_price(S0, K, r, sigma, T, N, M, 0, "C")
print("American ATM call price =", american_call_price)

American ATM call price = 4.58


Because there is no closed form solution to the **delta** and **vega** values of American options, we are going to use the **finite difference** method to calculate these greeks:

$\Delta \simeq \Large\frac{C(S_t + dS_t; \cdots) - C(S_t; \cdots)}{dS_t}$

$\nu \simeq \Large\frac{C(\sigma_t + d\sigma_t; \cdots) - C(\sigma_t; \cdots)}{d\sigma_t}$

where,

$\Delta:$ Delta value of the option, sensitivity of the option price to the small change in the **price** of the underlying asset.

$S_t:$ Price of the underlying stock at time $t$

$C(\cdots):$ Price of the option given some parameters $S_t, \sigma_t, \cdots$ at time $t$.

$\nu:$ Vega value of the option, sensitivity of the option price to the small change in the **volatility** of the underlying asset.

$dS_t:$ Small change in the price of the underlying at time $t$.

$d\sigma_t:$ Small change in the volatility of the underlying at time $t$.

In [60]:
def american_delta(S0, K, r, sigma, T, N, M, t, call_or_put="C", epsilon=0.01):
    delta = (MC_American_price(S0 + S0*epsilon, K, r, sigma, T, N, M, t, call_or_put) - MC_American_price(S0, K, r, sigma, T, N, M, t, call_or_put)) / (S0 * epsilon)
    return np.round(delta, 2)

def american_vega(S0, K, r, sigma, T, N, M, t, call_or_put="C", epsilon=0.01):
    vega = (MC_American_price(S0, K, r, sigma + sigma*epsilon, T, N, M, t, call_or_put) - MC_American_price(S0, K, r, sigma, T, N, M, t, call_or_put)) / (sigma * epsilon)
    return np.round(vega, 2)

In [61]:
american_call_delta = american_delta(S0, K, r, sigma, T, N, M, 0, "C")
american_call_vega = american_vega(S0, K, r, sigma, T, N, M, 0, "C")

print("American ATM call delta =", american_call_delta)
print("American ATM call vega =", american_call_vega)

American ATM call delta = 0.81
American ATM call vega = 15.0


# Step 3

## Team Member A (Atakan Devrent)

7)

European Call option with 110% moneyness: $S_0 = 100, K = 110$

European Put option with 95% moneyness: $S_0 = 100, K = 95$

Both have $T = 0.25$

In [62]:
S0 = 100
K_call = 110
r = 0.05
sigma = 0.2
T = 0.25
t = 0

call_110_price = BS_price(S0, K_call, r, sigma, T, t, "C")

S0 = 100
K_put = 95
r = 0.05
sigma = 0.2
T = 0.25

put_95_price = BS_price(S0, K_put, r, sigma, T, t, "P")

print("European Call with 110% moneyness price =", call_110_price)
print("European Put with 95% moneyness price =", put_95_price)

European Call with 110% moneyness price = 1.19
European Put with 95% moneyness price = 1.53


$\Delta_{call} = \Phi(d_1)$

$\Delta_{put} = \Phi(d_1) - 1$

And if we buy both the call and put option, delta of the portfolio is simply the sum of the delta of the options (because they have the same weight):

$\Delta_{portfolio} = \Delta_{call} + \Delta_{put}$

In [63]:
d1_call = (np.log(S0/K_call) + (r + sigma**2 / 2) * T) / (sigma * np.sqrt(T))
d1_put = (np.log(S0/K_put) + (r + sigma**2 / 2) * T) / (sigma * np.sqrt(T))

call_delta = np.round(ss.norm.cdf(d1_call), 2)
put_delta = np.round(ss.norm.cdf(d1_put) - 1, 2)
portfolio_delta = call_delta + put_delta

print("Total price to buy both options at t=0 :", np.round(call_110_price + put_95_price, 2))
print("Call delta =", call_delta)
print("Put delta =", put_delta)
print("Portfolio delta =", portfolio_delta)

Total price to buy both options at t=0 : 2.72
Call delta = 0.22
Put delta = -0.25
Portfolio delta = -0.03


Replication Portfolio Calculation:

$B + \Delta \times S_0 = P$

where,

$B:$ Risk-free bond buy amount

$S_0:$ Underlying asset price at $t=0$

$\Delta:$ Delta value of the option to the underlying

$P:$ Price of the option portfolio at $t=0$

Then to replicate this specific portfolio (buy both options), we have the equation:

$ B + -0.03 \times 100 = 2.72 $

$\rightarrow B = 5.72$

This means, to delta-hedge this portfolio:

* We **BUY** $0.03$ units of the underlying asset at $t=0$.
* We **BORROW** $5.72$ units of the risk-free bond at $t=0$.

_________________

For the second portfolio we **buy** the call but **sell** the put. Now because the sign of the put is reversed, its delta value is also reversed. So the delta of the new portfolio will be:

$\Delta_{portfolio} = \Delta_{call} - \Delta_{put}$

In [64]:
portfolio_2_delta = call_delta - put_delta
print("Total price to buy call and sell put at t=0 :", np.round(call_110_price - put_95_price, 2))
print("New portfolio delta =", portfolio_2_delta)

Total price to buy call and sell put at t=0 : -0.34
New portfolio delta = 0.47


To delta hedge this new portfolio, we have a new equation:

$B + 0.47 \times 100 = -0.34$

* We **SELL** $0.47$ units of the underlying asset at $t=0$.
* We **LEND** $-47.34$ units of the risk-free bond at $t=0$.